## pandas-profiling Meteorites example
Source of data: https://data.nasa.gov/Space-Science/Meteorite-Landings/gh4g-9sfh

### Import libraries

In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
from pathlib import Path

### Load and prepare example dataset
We add some fake variables for illustrating pandas-profiling capabilities

In [2]:
file_name = Path('rows.csv')
if not file_name.exists():
    df = requests.get('https://data.nasa.gov/api/views/gh4g-9sfh/rows.csv?accessType=DOWNLOAD')
    file_name.write_bytes(data.content)
    
df = pd.read_csv(file_name)
    
# Note: Pandas does not support dates before 1880, so we ignore these for this analysis
df['year'] = pd.to_datetime(df['year'], errors='coerce')

# Example: Constant variable
df['source'] = "NASA"

# Example: Boolean variable
df['boolean'] = np.random.choice([True, False], df.shape[0])

# Example: Mixed with base types
df['mixed'] = np.random.choice([1, "A"], df.shape[0])

# Example: Highly correlated variables
df['reclat_city'] = df['reclat'] + np.random.normal(scale=5,size=(len(df)))

# Example: Duplicate observations
duplicates_to_add = pd.DataFrame(df.iloc[0:10])
duplicates_to_add[u'name'] = duplicates_to_add[u'name'] + " copy"

df = df.append(duplicates_to_add, ignore_index=True)

### Inline report without saving object

In [3]:
pr = df.profile_report()
from IPython.core.display import HTML
HTML(pr.get_style())

In [4]:
pr

Number of variables,16
Number of observations,45726
Missing cells,110522 (15.1%)
Duplicate rows,0 (0.0%)
Total size in memory,5.3 MiB
Average record size in memory,121.0 B
Numeric,6
Categorical,5
Boolean,1
Date,1
Text (Unique),1


### Save report to file

In [5]:
pfr = df.profile_report()
pfr.to_file("/tmp/example.html")

#### Print existing ProfileReport object inline

In [6]:
pfr

Number of variables,16
Number of observations,45726
Missing cells,110522 (15.1%)
Duplicate rows,0 (0.0%)
Total size in memory,5.3 MiB
Average record size in memory,121.0 B
Numeric,6
Categorical,5
Boolean,1
Date,1
Text (Unique),1
